In [23]:
from mlflow import MlflowClient

from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"


In [24]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)



In [25]:
client.create_experiment(name="my-cool-experiment")

MlflowException: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool-experiment', None, 'active', 1681180683406, 1681180683406)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [26]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [27]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 8499b64b75e34d418f65a1a6ee9f9120, rmse: 6.3005
run id: 9ffafd2721c64e5293819e8a375ae1e8, rmse: 6.3021
run id: b145771252a04dc3924bd122e9b75772, rmse: 6.3024
run id: 5ad303ed24204a4785bb80b70e3c1b2e, rmse: 6.3024
run id: 5fdd35bee58c4de0a9085be4e7a28415, rmse: 6.3058


In [28]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [31]:
run_id = "b145771252a04dc3924bd122e9b75772"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/04/11 05:39:28 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1681180768382, current_stage='None', description=None, last_updated_timestamp=1681180768382, name='nyc-taxi-regressor', run_id='b145771252a04dc3924bd122e9b75772', run_link=None, source=('file:///d:/henry/Data '
 'Science/MLops/experiment-tracking/mlruns/1/b145771252a04dc3924bd122e9b75772/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [32]:
model_uri

'runs:/b145771252a04dc3924bd122e9b75772/model'

In [34]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: Production
version: 3, stage: None


In [36]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1681180768382, current_stage='Staging', description=None, last_updated_timestamp=1681182395088, name='nyc-taxi-regressor', run_id='b145771252a04dc3924bd122e9b75772', run_link=None, source=('file:///d:/henry/Data '
 'Science/MLops/experiment-tracking/mlruns/1/b145771252a04dc3924bd122e9b75772/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [37]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1681180768382, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-04-11', last_updated_timestamp=1681182398823, name='nyc-taxi-regressor', run_id='b145771252a04dc3924bd122e9b75772', run_link=None, source=('file:///d:/henry/Data '
 'Science/MLops/experiment-tracking/mlruns/1/b145771252a04dc3924bd122e9b75772/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [38]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [46]:
df = read_dataframe("data/green_tripdata_2021-03.csv")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23700\3050441246.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [47]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23700\2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'd:\\henry\\Data Science\\MLops\\experiment-tracking\\preprocessor'

In [48]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [49]:
X_test = preprocess(df, dv)

In [50]:
target = "duration"
y_test = df[target].values

In [51]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: total: 11.3 s
Wall time: 17.2 s


{'rmse': 6.885236608020716}

In [52]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

[06:28:50] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: total: 21.4 s
Wall time: 3.87 s


{'rmse': 6.248294328516825}

In [53]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1681180768382, current_stage='Production', description='The model version 3 was transitioned to Staging on 2023-04-11', last_updated_timestamp=1681183841125, name='nyc-taxi-regressor', run_id='b145771252a04dc3924bd122e9b75772', run_link=None, source=('file:///d:/henry/Data '
 'Science/MLops/experiment-tracking/mlruns/1/b145771252a04dc3924bd122e9b75772/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>